In [0]:
%python
# Define the file path
filePath = "/Volumes/workspace/globalretail/transaction_snappy"

# Read the JSON file into a DataFrame
df = spark.read.parquet(filePath)

# Display the DataFrame
display(df)

In [0]:
from pyspark.sql.functions import to_timestamp, col
new_df = df.withColumn("transaction_date", to_timestamp(col("transaction_date")))
new_df.printSchema()
display(new_df)

In [0]:
from pyspark.sql.functions import current_timestamp
final_df = new_df.withColumn("ingestion_timestamp", current_timestamp())
display(final_df)

In [0]:
spark.sql("use globalretail_bronze")
final_df.write.format("delta").mode("append").saveAsTable("bronze_transactions")

In [0]:
spark.sql("select * from bronze_transactions limit 100").show()

In [0]:
%python
# Create the volume if it does not exist
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.globalretail.archive
COMMENT 'Archive volume for transaction_snappy data'
""")

import datetime

# Define the source and archive paths
source_file = "/Volumes/workspace/globalretail/transaction_snappy/"
archive_folder = "/Volumes/workspace/globalretail/archive"
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
archive_file = f"{archive_folder}/transaction_snappy_{timestamp}.csv"

# Move the file from the original path to the new archive path
dbutils.fs.mv(source_file, archive_file, recurse=True)

# Print the new archive file path
print(f"File archived at: {archive_file}")